In [59]:
from sympy.logic.boolalg import to_anf,to_cnf,to_dnf,to_nnf,simplify_logic,truth_table
from collections import defaultdict
import re
dct = defaultdict(set)
TAUTOLOGY_RE = re.compile(r'~?([a-zA-Z]{1}) ?[|&] ?~?\1')


def _set_identities(fmt_str, name_bool_pairs:list):
    """
    `name_bool_pairs` is of the form `[('p', True), ('q', True)]`, so that `args` will be built as e.g. `['p', '~q']`,
    then formatted via `fmt_str`.
    """
    args = []
    
    for name,boolean in name_bool_pairs:
        args.append(name if boolean else f'~{name}')
    try:
        expr = fmt_str.format(*args)
    except IndexError:
        print(f'IndexError, {args=}')
        return
    try:
        anf = to_anf(expr)
    except TypeError:
        anf = ""
    try:
        cnf = to_cnf(expr)
    except TypeError:
        cnf = ""
    try:
        dnf = to_dnf(expr)
    except TypeError:
        dnf=""
    try:
        nnf = to_nnf(expr)
    except AttributeError:
        nnf = ""
    try:
        simple = str(simplify_logic(expr))
    except TypeError:
        simple=""
    _is_ok = lambda s: (
        bool(str(s))
        and 
        not '^' in s
        and 
        not TAUTOLOGY_RE.search(s)
        and 
        not s in [simple,expr_str]
        )
    
    if len(simple) > 2: # skip if simple == 'p' or simple == '~p'
        print(f'[{expr}] ≡ {simple}')
#         print(list(truth_table(expr,set(args))))
        if _is_ok(expr_str := str(expr)):
            dct[simple].add(expr)
        if _is_ok(str(anf)):
            print(f'\tanf: "{anf}"')
            dct[simple].add(anf)
        if _is_ok(str(cnf)):
            print(f'\tcnf: "{cnf}"')
            dct[simple].add(cnf)
        if _is_ok(str(dnf)):
            print(f'\tdnf: "{dnf}"')
            dct[simple].add(dnf)
        if _is_ok(str(nnf)):
            print(f'\tnnf: "{nnf}"')
            dct[simple].add(nnf)
        print()
    

def set_identities_ij(fmt_str, *symbols):
    if len(symbols) != 2:
        raise ValueError(f"because ij, expecting 2 symbols. got {symbols=}")
    for i in (True, False):
        for j in (True, False):
            pairs = zip(symbols, (i,j))
            _set_identities(fmt_str, pairs)

        
def set_identities_ijk(fmt_str, *symbols):
    if len(symbols) != 3:
        raise ValueError(f"because ijk, expecting 3 symbols. got {symbols=}")
    for i in (True, False):
        for j in (True, False):
            for k in (True, False):
                pairs = zip(symbols, (i,j,k))
                _set_identities(fmt_str, pairs)
def set_identities_ijkl(fmt_str, *symbols):
    if len(symbols) != 4:
        raise ValueError(f"because ijkl, expecting 4 symbols. got {symbols=}")
    for i in (True, False):
        for j in (True, False):
            for k in (True, False):
                for l in (True, False):
                    pairs = zip(symbols, (i,j,k,l))
                    _set_identities(fmt_str, pairs)
                
                



In [ ]:
p_or_q__and_p = set_identities('({} | {}) & {}', 'p','q','p')
p_and_q__or_p = set_identities('({} & {}) | {}', 'p','q','p')
p_or_q__and_q = set_identities('({} | {}) & {}','p','q','q')
p_and_q__or_q = set_identities('({} & {}) | {}','p','q','q')

In [61]:
set_identities_ijk('{} >> ({} & {})', 'p','q', 'r')

[p >> (q & r)] ≡ ~p | (q & r)
	cnf: "(q | ~p) & (r | ~p)"

[p >> (q & ~r)] ≡ ~p | (q & ~r)
	cnf: "(q | ~p) & (~p | ~r)"

[p >> (~q & r)] ≡ ~p | (r & ~q)
	cnf: "(r | ~p) & (~p | ~q)"

[p >> (~q & ~r)] ≡ ~p | (~q & ~r)
	cnf: "(~p | ~q) & (~p | ~r)"

[~p >> (q & r)] ≡ p | (q & r)
	cnf: "(p | q) & (p | r)"

[~p >> (q & ~r)] ≡ p | (q & ~r)
	cnf: "(p | q) & (p | ~r)"

[~p >> (~q & r)] ≡ p | (r & ~q)
	cnf: "(p | r) & (p | ~q)"

[~p >> (~q & ~r)] ≡ p | (~q & ~r)
	cnf: "(p | ~q) & (p | ~r)"



In [53]:
set_identities_ij('{} & {}', 'p','q')

[p & q] ≡ p & q

[p & ~q] ≡ p & ~q
	anf: "p ^ (p & q)"

[~p & q] ≡ q & ~p
	anf: "q ^ (p & q)"

[~p & ~q] ≡ ~p & ~q
	anf: "p ^ q ^ True ^ (p & q)"



In [ ]:
set_identities('{} >> {} | {} >> {}', 'p','q','p','r')